In [1]:
import numpy as np

import sys
sys.path.append('../../bstadt/Quality/')
sys.path.append('../../bstadt/Util/')

from Quality import compute_overlap_array as bstadt_overlaps
from Util import generateTestVolume
from skimage.measure import label

In [2]:
data = np.load('../data/collman15v2.npz')

In [3]:
data.files

['annotation',
 'EM25K',
 'GABA488',
 'GAD647',
 'gephyrin594',
 'GS594',
 'MBP488',
 'NR1594',
 'PSD95_488',
 'Synapsin647',
 'VGluT1_647',
 'DAPI1st']

In [4]:
annotation = data['annotation']

### Check results from bstadt compute_overlap_array to make sure it is creating more ground truth labels

In [5]:
#Just look at the middle section with shape (20, 2000, 2000)
annotation = annotation[0:20, 2000:4000, 2000:4000]

In [6]:
#This section has 37 annotations
print(len(np.unique(annotation)[1:]))

#Labeling annotations results in 52 labels
print(len(np.unique(label(annotation))[1:]))

37
52


In [7]:
#Generate a random "prediction" volume with shape (20, 2000, 2000) with 40 predictions
test_volume, _ = generateTestVolume(20, 2000, 2000, 
                                40,
                                41,
                                3,
                                5,
                                10,
                                25,
                                10,
                                25)

In [15]:
#Confirm there are 40 "predictions"
len(np.unique(label(test_volume))[1:])

40

In [10]:
bstadt_results = bstadt_overlaps(test_volume, annotation)

predictionPerGt should have 52 values in bstadt_results

In [11]:
for key, value in bstadt_results.items():
    print(key, len(value))

predictionPerGt 52
gtPerPrediction 40


### Remove labeling of ground truth. Should result in "predictionPerGt" to be 37 in this case.

In [16]:
def get_uniques(ar):
    """
    Returns an ordered numpy array of unique integers in an array.
    This runs about four times faster than numpy.unique().

    Parameters
    ----------
    ar : array_like

    Returns
    -------
    uniques :
    """
    bins = np.zeros(np.max(ar) + 1, dtype=int)
    bins[ar.ravel()] = 1
    uniques = np.nonzero(bins)[0]

    return uniques

def get_unique_overlap2(foreground, background, i):
    '''
    Calculates the number of unique background labels in the foreground at i
    Does not count background label of 0
    '''

    overlaps = np.multiply((foreground == i), background)
    uniques = get_uniques(overlaps)

    num_unique = len(uniques)

    #0 is background label
    #should not count as a detection if
    #the prediction overlaps with the background
    if 0 in uniques:
        num_unique -= 1

    return num_unique


def compute_overlap_array2(predictions, gt):
    predictionLabels = label(predictions)
    maxPredictionLabel = np.max(predictionLabels)

    #gtLabels = label(gt)
    #maxGtLabel = np.max(gtLabels)
    gt_uniques = get_uniques(gt)[1:]


    #first, look at how many unique predictions
    #overlap with a single gt synapse
    predictionPerGt = [get_unique_overlap2(gt, predictionLabels, i)\
                       for i in gt_uniques]


    #next, look at how many unique synapses overlap
    #with a single synapse prediction
    gtPerPrediction = [get_unique_overlap2(predictionLabels, gt, i)\
                       for i in range(1, maxPredictionLabel + 1)]

    return {'predictionPerGt': predictionPerGt,
            'gtPerPrediction': gtPerPrediction}


In [18]:
j1c_results = compute_overlap_array2(test_volume, annotation)

This should result in predictionPerGt with 37 values

In [21]:
for key, value in j1c_results.items():
    print(key, len(value))

predictionPerGt 37
gtPerPrediction 40


In [25]:
np.array_equiv(get_uniques(annotation), np.unique(annotation))

True